In [1]:
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

In [26]:
data=pd.read_table('ck_data.txt',names=['y','X'])
data

,y,X
0,0,노래가 너무 적음
1,0,"돌겠네 진짜. 황숙아, 어크 공장 그만 돌려라. 죽는다."
2,1,막노동 체험판 막노동 하는사람인데 장비를 내가 사야돼 뭐지
3,1,차악!차악!!차악!!! 정말 이래서 왕국을 되찾을 수 있는거야??
4,1,시간 때우기에 좋음.. 도전과제는 50시간이면 다 깰 수 있어요
...,...,...
99995,0,한글화해주면 10개산다
99996,0,개쌉노잼 ㅋㅋ
99997,0,노잼이네요... 30분하고 지웠어요...
99998,1,야생을 사랑하는 사람들을 위한 짧지만 여운이 남는 이야기. 영어는 그리 어렵지 않습니다.


In [4]:
s_w=set(['은','는','이','가','를','들','에게','의','을','도','으로','만','라서','하다'])#불용어 정의
#s_w.add(불용어 추가문자열)#추가할 불용어는 add를 이용하여 입력
okt=Okt()#형태소 분석기(자율)

In [5]:
data.isnull().values.any()#결측치 확인

False

In [6]:
data['X'].nunique(), data['y'].nunique()#중복확인

(99892, 2)

In [7]:
data=data.drop_duplicates(subset=['X'])#중복제거
data['clean_X']=data.X.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')#한글외 정리 
data['clean_X']=data.clean_X.str.replace('^ +','')#공백 시작문자 정리
data['clean_X']=data.clean_X.replace('',np.nan) #공백 NaN화
data=data.dropna(how='any') #NaN데이터 정리

C:\Users\ho316\AppData\Local\Temp/ipykernel_10092/3642056730.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['clean_X']=data.X.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')#한글외 정리
C:\Users\ho316\AppData\Local\Temp/ipykernel_10092/3642056730.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['clean_X']=data.clean_X.str.replace('^ +','')#공백 시작문자 정리


In [8]:
okt=Okt()
from tqdm import tqdm 
X_data=[] 
for i in tqdm(data['clean_X']): 
    tk_d=okt.morphs(i) 
    end_d=[w for w in tk_d if not w in s_w]#불용어 처리 
    X_data.append(' '.join(end_d))

100%|███████████████████████████████████████████████████████████████████████████| 99892/99892 [03:35<00:00, 462.70it/s]


In [9]:
data['y_name']=data['y']#정답 이름 기록
data['encoder_y']=LabelEncoder().fit_transform(data['y'])#정답 숫자화
data['categorical_y']=list(to_categorical(data['encoder_y']))#다중 정답을 위한 희소행렬화

In [10]:
Y=np.array(data['encoder_y'])#이진

In [11]:
X=np.array(X_data)#입력정리

In [12]:
x_data,test_x,y_data,test_y = train_test_split(X,Y,test_size=0.3,random_state=0,stratify = Y)#태스트데이터 분할

In [13]:
train_x,val_x,train_y,val_y = train_test_split(x_data,y_data,test_size=0.2,random_state=0,stratify = y_data)#학습, 검증 데이터 분할

In [14]:
tk = Tokenizer()
tk.fit_on_texts(train_x)#문서의 단어수 확인

In [15]:
n=len([d for d in sorted(list(tk.word_counts.items()),key=lambda x:x[1]) if d[1]>4])+1#4번이하 입력단어 정리시 길이확인

In [16]:
token=Tokenizer(n)#고정된 단어수로 정리
token.fit_on_texts(train_x)

In [17]:
token_train_x=token.texts_to_sequences(train_x)
token_test_x=token.texts_to_sequences(test_x)
token_val_x=token.texts_to_sequences(val_x)

In [18]:
drop_train = [index for index, sentence in enumerate(token_train_x) if len(sentence) < 1]#빈 문장 위치 정보 확인
drop_test = [index for index, sentence in enumerate(token_test_x) if len(sentence) < 1]#빈 문장 위치 정보 확인
drop_val = [index for index, sentence in enumerate(token_val_x) if len(sentence) < 1]#빈 문장 위치 정보 확인

In [19]:
token_train_x = np.delete(token_train_x, drop_train, axis=0)#빈 문장 정리
train_y = np.delete(train_y, drop_train, axis=0)#빈 문장 정리
token_test_x = np.delete(token_test_x, drop_test, axis=0)#빈 문장 정리
test_y = np.delete(test_y, drop_test, axis=0)#빈 문장 정리
token_val_x = np.delete(token_val_x, drop_val, axis=0)#빈 문장 정리
val_y = np.delete(val_y, drop_val, axis=0)#빈 문장 정리

C:\Users\ho316\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [20]:
w_l=len(pad_sequences(token_train_x)[0])#문장 길이 확인

In [21]:
train_inputs = pad_sequences(token_train_x,maxlen=w_l)#동일 문장길이화
test_inputs = pad_sequences(token_test_x,maxlen=w_l)#동일 문장길이화
val_inputs = pad_sequences(token_val_x,maxlen=w_l)#동일 문장길이화

In [22]:
train_outputs=train_y#출력정리
test_outputs=test_y#출력정리
val_outputs=val_y#출력정리

In [23]:
전처리_된_데이터={}
전처리_된_데이터['학습_데이터']=train_inputs
전처리_된_데이터['학습_결과']=train_outputs
전처리_된_데이터['태스트_데이터']=test_inputs
전처리_된_데이터['태스트_결과']=test_outputs
전처리_된_데이터['검증_데이터']=val_inputs
전처리_된_데이터['검증_결과']=val_outputs
전처리_된_데이터['토큰']=token
전처리_된_데이터['단어수']=n
전처리_된_데이터['불용어']=s_w
전처리_된_데이터['입력길이']=w_l

In [24]:
전처리_된_데이터.keys()

dict_keys(['학습_데이터', '학습_결과', '태스트_데이터', '태스트_결과', '검증_데이터', '검증_결과', '토큰', '단어수', '불용어', '입력길이'])

In [25]:
import pickle
with open("data1.pickle","wb") as fw:
    pickle.dump(전처리_된_데이터, fw)